# Example of ML pipeline and grid optimization

Code and data data fro Udacity Nanodegree class.

### 1.  Load libraries

In [1]:
from pyspark.sql.functions import avg, col, concat, desc, explode, lit, min, max, split, udf
from pyspark.sql.types import IntegerType

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import CountVectorizer, IDF, Normalizer, PCA, RegexTokenizer, StandardScaler, StopWordsRemover, StringIndexer, VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

import re
import time

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1623860404582_0002,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 2. Load and persist data

In [2]:
stack_overflow_data = "s3n://sparkify-mr/data/Train_onetag_small.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df2 = spark.read.json(stack_overflow_data)
df2.persist()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[Body: string, Id: bigint, Tags: string, Title: string, oneTag: string]

### 3. Split in training and test dataset

In [4]:
# 90/10 % split
train, test = df2.randomSplit([0.9, 0.1], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 4. Define pipeline

In [5]:
# The next three create the features
# Tokenizer
regexTokenizer = RegexTokenizer(inputCol="Body", outputCol="words", pattern="\\W")
# Vectorizer
cv = CountVectorizer(inputCol="words", outputCol="TF", vocabSize=10000)
# IDF Transformer
idf = IDF(inputCol="TF", outputCol="features")

# This creates the label
# Indexer
indexer = StringIndexer(inputCol="oneTag", outputCol="label")

# Logistic regression model
lr =  LogisticRegression(maxIter=10, regParam=0.0, elasticNetParam=0)

# Set the pipeline
# Note, the first three HAVE to be in that order, indexer could have been elsewhere
pipeline = Pipeline(stages=[regexTokenizer, cv, idf, indexer, lr])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 5. Define optimization grid and cross-validator

On the first 90% of the data let's find the most accurate logistic regression model using 3-fold cross-validation with the following parameter grid:

- CountVectorizer vocabulary size: `[1000, 5000]`
- LogisticRegression regularization parameter: `[0.0, 0.1]`
- LogisticRegression max Iteration number: `[10]`

In [6]:
paramGrid = ParamGridBuilder() \
    .addGrid(cv.vocabSize,[1000, 5000]) \
    .addGrid(lr.regParam,[0.0, 0.1]) \
    .build()


crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=MulticlassClassificationEvaluator(),
                          numFolds=3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 6. Train

In [7]:
# Fit
cvModel_q1 = crossval.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### 7. Evaluate

In [8]:
# Average metrics
cvModel_q1.avgMetrics

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0.3009102619501403, 0.2327996664299532, 0.3619649667272708, 0.28220776028385464]

In [9]:
# Prediction
results = cvModel_q1.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Take a look at the results
print(results.filter(results.label == results.prediction).count())
print(results.count())
print(results.filter(results.label == results.prediction).count()/results.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

3897
9937
0.3921706752541008

So we get correct results in ~ 40% cases